In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error , r2_score, accuracy_score
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt 
from sklearn.tree import DecisionTreeClassifier
import json
from pypmml import Model
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline
import pickle

In [3]:
df = pd.read_csv('adult.data', sep=',' , names = [
    'age',
    'workclass',
    'fnlwgt',
    'education',
    'education-num',
    'marital-status',
    'occupation',
    'relationship',
    'race',
    'sex',
    'capital-gain',
    'capital-loss',
    'hours-per-week',
    'native-country',
    'income'])

In [4]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]

In [5]:
X.columns

Index(['age', 'workclass', 'fnlwgt', 'education', 'education-num',
       'marital-status', 'occupation', 'relationship', 'race', 'sex',
       'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'],
      dtype='object')

In [6]:
X.iloc[0,:]

age                           39
workclass              State-gov
fnlwgt                     77516
education              Bachelors
education-num                 13
marital-status     Never-married
occupation          Adm-clerical
relationship       Not-in-family
race                       White
sex                         Male
capital-gain                2174
capital-loss                   0
hours-per-week                40
native-country     United-States
Name: 0, dtype: object

In [7]:
label_encoder = LabelEncoder()
for col in X.columns:
    if type(X.loc[0 , col]) == str:
        X.loc[: , col] = label_encoder.fit_transform(X.loc[:,col])


/tmp/ipykernel_6852/3916783739.py:4: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  X.loc[: , col] = label_encoder.fit_transform(X.loc[:,col])


In [8]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [9]:
Y = label_encoder.fit_transform(Y)

In [17]:
output = open('Departure_encoder.pkl', 'wb')
pickle.dump(label_encoder, output)
output.close()

In [10]:
X_train , X_test , Y_train, Y_test = train_test_split(X,Y , test_size=0.2)


In [11]:
model = DecisionTreeClassifier()


pipeline = PMMLPipeline([
	("classifier", DecisionTreeClassifier())
])
pipeline.fit(X_train, Y_train)

# Save the fitted model
sklearn2pmml(pipeline, "dtr.pmml", with_repr = True)

# Load the saved model and predict data
model = Model.fromFile('dtr.pmml')


/home/abolfazl1381/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn2pmml/pipeline/__init__.py:68: UserWarning: X is missing feature names. The reproducibility of predictions between Scikit-Learn and PMML can not be guaranteed
  warnings.warn("X is missing feature names. The reproducibility of predictions between Scikit-Learn and PMML can not be guaranteed")
/home/abolfazl1381/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn2pmml/pipeline/__init__.py:75: UserWarning: y is missing target field name(s)
  warnings.warn("y is missing target field name(s)")


In [12]:
Y_test

array([1, 1, 0, ..., 0, 0, 0])

In [13]:
y_pred

NameError: name 'y_pred' is not defined

In [ ]:
y_pred = pd.DataFrame(y_pred)


In [ ]:
Y_test = label_encoder.inverse_transform(Y_test)

/home/abolfazl1381/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
y_pred = label_encoder.inverse_transform(y_pred)

/home/abolfazl1381/anaconda3/envs/tf-gpu/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:154: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
Y_test = pd.DataFrame(Y_test)
y_pred = pd.DataFrame(y_pred)

In [ ]:
total_answer = pd.concat([y_pred , Y_test])

/tmp/ipykernel_6021/1436144399.py:1: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  total_answer = pd.concat(y_pred , Y_test)


TypeError: first argument must be an iterable of pandas objects, you passed an object of type "DataFrame"